# 활동여부크롤러 제작중

In [ ]:
import module.chrome_driver_module as chrome_driver_module
import module.db_module as db_module
import pandas as pd
import numpy as np

from module.activate_seller.naver_jungo.main import naver_jungo
from module.activate_seller.bunjang.main import bunjang
from module.db_module import CreateTableIfNotExists
from sqlalchemy import Table, Column, Integer, String, Boolean
from module.activate_seller.rearrange_as_id import rearrange_as_id

db_class = db_module.Database() #db연결 생성

######### activate_seller 테이블 없다면 정의해주는 부분 ###########
table = Table(
    "activate_seller",
    db_class.metadata,
    Column("as_id", Integer, primary_key=True),
    Column("vendor_type", String(50)),
    Column("fsl_rs_id", String(50)),
    Column("shop_id", String(50)),
    Column("last_sell", String(50)),
    Column("sell_count", String(50)),
    Column("dealing_smartphone", String(50)),
    Column("is_activate", Boolean),
    mysql_auto_increment='0',
)
db_class.engine.execute(CreateTableIfNotExists(table))


driver = chrome_driver_module.ChromeDriver().driver

   #############################################
  #        워닛 판매자 활동여부 크롤링         #
 #  입력: ? / 출력: activate_seller 테이블    #
#############################################

# 결과데이터 형식 정의
my_dict = {
    "vendor_type" : "", 
    "fsl_rs_id" : "",               #리얼셀러 아이디
    "shop_id":"",                   #네이버/번장 판매자 아이디
    "last_sell": "",                #가장최근 판매글 게시일
    "sell_count": "",               #2주간 판매중인 물품 갯수
    "dealing_smartphone": "",       #2주간 스마트폰 품목 취급여부
    "is_activate": ""               #종합적으로 판단한 최종 활동여부
}
crawl_data = pd.DataFrame(my_dict,index=[0])

seller_list = pd.read_excel('셀러리스트.xlsx').fillna(np.NAN) #입력데이터

# for i in range(len(seller_list)):
for i in range(3): #테스트용
    
    #네이버판매처 크롤링
    crawl_data = naver_jungo(crawl_data,seller_list,i,driver)

    #번개장터판매처 크롤링
    crawl_data = bunjang(crawl_data,seller_list,i,driver)

crawl_data = crawl_data.reset_index()
indexNames = crawl_data[crawl_data['vendor_type'] == "" ].index
crawl_data.drop(indexNames,inplace=True)
crawl_data = crawl_data.drop(['index'], axis='columns')

activate_seller = pd.read_sql_table('activate_seller',db_class.engine_conn).drop(['as_id'], axis='columns') #기존테이블 로드
crawl_data = crawl_data.append(activate_seller, ignore_index=True)
crawl_data = crawl_data.drop_duplicates(['fsl_rs_id','shop_id'], keep='last')


try:    
    with db_class.cursor as cursor:
        cursor.execute("DELETE FROM activate_seller;")
        db_class.db_conn.commit()
        
finally:
     db_class.db_conn.close()

crawl_data.to_sql(name='activate_seller', con=db_class.engine, if_exists='append', index=False)
db_class.engine_conn.close()
    
rearrange_as_id()




In [ ]:
crawl_data

# ORM 테스트

In [1]:
import module.db_module as db_module
db_class = db_module.Database() #db연결 생성


In [2]:
import sqlalchemy
sqlalchemy.__version__ 

'1.4.7'

In [3]:
engine = db_class.engine

In [4]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [5]:
from sqlalchemy import Column, Integer, String
class User(Base):
        __tablename__ = 'users'

        id = Column(Integer, primary_key=True)
        name = Column(String(30))
        fullname = Column(String(30))
        nickname = Column(String(30))
    
        def __repr__(self):
            return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)

In [6]:
User.__table__ 

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(length=30), table=<users>), Column('fullname', String(length=30), table=<users>), Column('nickname', String(length=30), table=<users>), schema=None)

In [7]:
Base.metadata.create_all(engine)

In [8]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [9]:
ed_user.name

'ed'

In [10]:
ed_user.nickname

'edsnickname'

In [11]:
str(ed_user.id)

'None'

In [12]:
from sqlalchemy.orm import sessionmaker

In [13]:
Session = sessionmaker(bind=engine)

In [16]:
session = Session()

In [17]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [18]:
session.add(ed_user)

In [19]:
our_user = session.query(User).filter_by(name='ed').first() 

In [20]:
our_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [21]:
ed_user is our_user

True

In [22]:
session.add_all([
         User(name='wendy', fullname='Wendy Williams', nickname='windy'),
         User(name='mary', fullname='Mary Contrary', nickname='mary'),
         User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [23]:
ed_user.nickname = 'eddie'

In [24]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [25]:
session.new  

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [26]:
session.commit()

In [27]:
ed_user.id 

1

In [28]:
ed_user.name = 'Edwardo'

In [29]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')

In [30]:
session.add(fake_user)

In [31]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [32]:
session.rollback() # 가장 최근의 커밋지점으로 롤백된다

In [33]:
ed_user.name

'ed'

In [34]:
fake_user in session

False

In [35]:
for instance in session.query(User).order_by(User.id):
        print(instance.name, instance.fullname)

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [36]:
for name, fullname in session.query(User.name, User.fullname):
         print(name, fullname)

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [37]:
for row in session.query(User, User.name).all():
        print(row.User, row.name)

<User(name='ed', fullname='Ed Jones', nickname='eddie')> ed
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> wendy
<User(name='mary', fullname='Mary Contrary', nickname='mary')> mary
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> fred


In [38]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

ed
wendy
mary
fred
